# SETUP para Google Collab


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive/genomica-2020-pf

In [ ]:
! git clone https://github.com/cpimentelGH/genomica_computacional_proyecto_final.git

In [ ]:
! ls

In [ ]:
# %cd genomica_computacional_proyecto_final/
! git checkout CNN-model

In [ ]:
%cd scripts/

In [ ]:
from CGRfuncs import to_cgr
from CGRfuncs2 import complexCGRGen

In [ ]:
# ! mkdir ../data/filtered

Creación del conjunto de entrenamiento

In [ ]:
# complexCGRGen('../data/raw_data/train/alpha/alpha.fasta', '../data/filtered/train/alpha', 'alpha')
# complexCGRGen('../data/raw_data/train/beta/beta.fasta', '../data/filtered/train/beta', 'beta')
# complexCGRGen('../data/raw_data/train/delta/delta.fasta', '../data/filtered/train/delta', 'delta')
# complexCGRGen('../data/raw_data/train/gamma/gamma.fasta', '../data/filtered/train/gamma', 'gamma')

Creación del conjunto de prueba

In [ ]:
# complexCGRGen('../data/raw_data/test/alpha.fasta', '../data/filtered/test/alpha', 'alpha')
# complexCGRGen('../data/raw_data/test/beta.fasta', '../data/filtered/test/beta', 'beta')
# complexCGRGen('../data/raw_data/test/delta.fasta', '../data/filtered/test/delta', 'delta')
# complexCGRGen('../data/raw_data/test/gamma.fasta', '../data/filtered/test/gamma', 'gamma')

# Creación del Modelo

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
img_width, img_height = 200, 200

train_data_dir = '../data/filtered/train'
validation_data_dir = '../data/filtered/test'
nb_train_samples = 1596
nb_validation_samples = 660
epochs = 10
batch_size = 10

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '../data/filtered/train',
    target_size=(200, 200),
    batch_size=10,
    class_mode='categorical')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    '../data/filtered/test',
    target_size=(200, 200),
    batch_size=10,
    class_mode='categorical')

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

# Prueba con Sars COV 2 

In [ ]:
import numpy as np
from keras.preprocessing import image

In [ ]:
to_cgr('../data/raw_data/sarscov2.fasta', '../data/filtered', 'sarscov2')

In [ ]:
img = image.load_img('../data/filtered/sarscov2.png', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [ ]:
images = np.vstack([x])
classes = model.predict_classes(images, batch_size=3)
print(classes[0])